In [2]:
import json
from datasets import load_from_disk, load_dataset
from transformers import RobertaTokenizer
from process_java_model import *
import os


In [3]:
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")


In [ ]:
DATASET_PATH = "../dataset/codesearchnet-java-discovered/"

data_files = {"train": [], "valid": [], "test": []}

for file in os.listdir(DATASET_PATH):
    file_path = os.path.join(DATASET_PATH, file)
    if "train" in file:
        data_files["train"].append(file_path)
    elif "valid" in file:
        data_files["valid"].append(file_path)
    elif "test" in file:
        data_files["test"].append(file_path)

print(data_files)

dataset = load_dataset("json", data_files=data_files)
print(dataset)


In [ ]:
def preprocess_examples(examples):
    contents = examples["code"]
    model_inputs = tokenizer(contents)
    return model_inputs


dataset_with_input = dataset.map(
    preprocess_examples,
    batched=True,
    batch_size=100,
    num_proc=64,
)


In [ ]:
def preprocess_examples(examples):
    xmi = examples["xmi"]

    seqs = [generate_sequence(xmi_string) for xmi_string in xmi]
    seqs = [json.dumps(seq) for seq in seqs]

    labels = tokenizer(seqs).input_ids

    return {"seq": seqs, "labels": labels}


dataset_with_output = dataset_with_input.map(
    preprocess_examples,
    batched=True,
    batch_size=10,
    num_proc=64,
)


In [7]:
print(dataset_with_output)
dataset_with_output.save_to_disk("/data/nicolasmaier/dataset/hf_seq_dataset_2")


DatasetDict({
    train: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'seq', 'labels'],
        num_rows: 454273
    })
    valid: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'seq', 'labels'],
        num_rows: 15326
    })
    test: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'seq', 'labels'],
        num_rows: 26902
    })
})


In [ ]:
MAX_LENGTH = 505

dataset_filtered = dataset_with_output.filter(
    lambda example: len(example["input_ids"]) <= MAX_LENGTH, num_proc=64
).filter(
    lambda example: len(example["seq"]) >= 5 and len(example["labels"]) <= MAX_LENGTH,
    num_proc=64,
)


In [9]:
print(dataset_filtered)
dataset_filtered.save_to_disk("/data/nicolasmaier/dataset/hf_clean_seq_dataset_2")


DatasetDict({
    train: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'seq', 'labels'],
        num_rows: 381183
    })
    valid: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'seq', 'labels'],
        num_rows: 13383
    })
    test: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'seq', 'labels'],
        num_rows: 22333
    })
})


Flattening the indices:   0%|          | 0/382 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/14 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/23 [00:00<?, ?ba/s]